# DLAD Exercise 2: Multitask Learning

### Working with code

- fork the original exercise template repository under your user account, make sure that the fork remains private
- [Set up an "upstream" remote of your fork](https://help.github.com/en/github/collaborating-with-issues-and-pull-requests/configuring-a-remote-for-a-fork) so that you can [pull in changes from the upstream repository](https://help.github.com/en/github/collaborating-with-issues-and-pull-requests/syncing-a-fork) (template solution you forked) whenever we make important updates (announced via Piazza/email)
- add your fork to SageMaker repositories (see course presentation on AWS or check [here](https://aws.amazon.com/blogs/machine-learning/amazon-sagemaker-notebooks-now-support-git-integration-for-increased-persistence-collaboration-and-reproducibility/))
- clone your fork to your PC, edit code, push changes back to your fork

### Running experiments

- The code is pulled from a linked repository _automatically_ only when you start the Notebook Instance. Subsequent pulls have to be done from the Terminal (see below)
- The only cell in this notebook launches a new Training Job, and trains a model with it until the end
- You can watch live TensorBoard every now and then to make sure training is doing fine. Just making sure the first training steps did not fail is enough in most cases
- If you regret the hyperparameter choice, you can stop the job by visiting [SageMaker Training Jobs Console](https://console.aws.amazon.com/sagemaker/home?region=us-east-1#/jobs), selecting the job in progress, and pressing Stop button. The job will take a couple of minutes to transition into Stopped state; until then your job is counted against your limits. After that you can stop the notebook cell. *Stopping the notebook cell alone does not terminate the Training Job, but only disconnects the notebook from it*
- Each Training Job has a very simple lifecycle: it either runs until the end, thus resulting in a `submission.zip` file, or it fails/stops, in which case it cannot be resumed.

### Incremental changes

- (optional) With code changes
 - change code in your standalone environment
 - dry run code in your standalone environment to make sure it doesn't fail due to a syntax error, thus saving you time with Training Job initialization (order of 5 minutes)
 - push code to your private fork
 - open the Terminal within your Notebook Instance (Jupyter Logo in top-left corner -> New -> Terminal), navigate to your code folder
 - execute `git pull` to fetch the changes you pushed to your fork
- (optional) With hyperparameter changes
 - update the hyperparameters dictionary in the cell below. It contains the command line parameters, which you would normally pass via command line as `--key <value>` in the standalone environment. All command line parameters are described in `mtl/utils/config.py`.
- Assign a meaningful yet short name to your experiment. A good name allows you to quickly understand the meaning of the experiment when looking at a list with 20 other experiments. Example: `question1-lr-0.001-optimizer-adam`
- Run the cell

### Running multiple experiments simultaneously

Each account is likely to have a limit of ml.p2.xlarge instances larger than 1, meaning that training multiple experiments in parallel is possible. To do that, we recommend having multiple (equal to your limit) separate Notebook Instances, each linked with the same repository. This way each of the Notebook Instances corresponds to one Training Job at time.

## Start a Training Job

- "In [\*]" next to the cell means the Training Job is running
- "In [any digit]" means the Trainin Job has finished, or the cell was disconnected from the job (either manually or by restarting/reconnecting to the notebook)

In [43]:
hyperparameters={
    # enable tensorboard server to run alongside the training code (you need this to track progress)
    'tensorboard_daemon_start': True,
    # enable forwarding tensorboard to your ngrok.com account (you need this to access tensorboard)
    'ngrok_daemon_start': True,
    # TODO: use your personal ngrok.com account authtoken to access tensorboard
    'ngrok_auth_token': '1afOlP4NIiVdFYkxUIrEctjBL0O_4NpsyRhTMoQt1zdTXugyN',
    
    # TODO: override configurable experiment parameters below this line
    # 'batch_size': <new value>,
    # 'num_epochs': ...,
    # ...
    'optimizer': 'adam',
    'optimizer_lr': 0.0001,
    'batch_size': 16,
    'loss_weight_semseg': 0.6,
    'loss_weight_depth': 0.4,
    'model_name': 'distillation',
    'aug_input_crop_size': 512    
    #'aug_geom_scale_min': 0.5,
    #'aug_geom_scale_max': 1.0,
    #'aug_geom_tilt_max_deg': 0.33,
    #'aug_geom_wiggle_max_ratio': 0.33,
    #'aug_geom_reflect': True,

}

# TODO: give name to distinct experiments, e.g. 'batch-size-4--newideaX-on--someparam-off'
experiment_name = 'Problem4-cropsize512'

#
# no changes below this line
#
import datetime
import sagemaker
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    source_dir='./',
    entry_point='train_sagemaker.py',
    role=sagemaker.get_execution_role(),
    framework_version='1.4.0',
    train_instance_count=1,
    train_instance_type='ml.p2.xlarge',
    train_volume_size=30,
    train_use_spot_instances=True,
    train_max_run=86000,
    train_max_wait=86400,
    debugger_hook_config=False,
    hyperparameters=hyperparameters,
)

estimator.fit(
    {'training': 's3://dlad-miniscapes/miniscapes.zip'},
    job_name=experiment_name + datetime.datetime.now().strftime("-%Y-%m-%d-%H-%M-%S"),
)

2020-05-27 09:41:41 Starting - Starting the training job...
2020-05-27 09:41:44 Starting - Launching requested ML instances.........
2020-05-27 09:43:28 Starting - Preparing the instances for training.........
2020-05-27 09:44:43 Downloading - Downloading input data.........
2020-05-27 09:46:18 Training - Downloading the training image........bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-05-27 09:47:53,649 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-05-27 09:47:53,682 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-05-27 09:47:55,106 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-05-27 09:47:56,062 sagemaker-containers INFO     Installing module with the following command:
/opt/conda/bin/python -m pip install . -r requirements.txt
Processing /tmp/tmpapgwp6le/module_dir

2020-05-27 09:47:52 T

{
    "aug_geom_reflect": false,
    "aug_geom_scale_max": 1.0,
    "aug_geom_scale_min": 1.0,
    "aug_geom_tilt_max_deg": 0.0,
    "aug_geom_wiggle_max_ratio": 0.0,
    "aug_input_crop_size": 512,
    "batch_size": 16,
    "batch_size_validation": 8,
    "dataset": "miniscapes",
    "dataset_root": "/opt/ml/input/data/training/miniscapes",
    "log_dir": "/opt/ml/model/log",
    "log_to_console": true,
    "loss_weight_depth": 0.4,
    "loss_weight_semseg": 0.6,
    "lr_scheduler": "poly",
    "lr_scheduler_power": 0.9,
    "model_encoder_name": "resnet34",
    "model_name": "distillation",
    "ngrok_auth_token": "1afOlP4NIiVdFYkxUIrEctjBL0O_4NpsyRhTMoQt1zdTXugyN",
    "ngrok_daemon_start": true,
    "num_epochs": 16,
    "num_steps_visualization_first": 100,
    "num_steps_visualization_interval": 1000,
    "observe_train_ids": [
        0,
        100
    ],
    "observe_valid_ids": [
        0,
        100
    ],
    "optimizer": "adam",
    "optimizer_lr": 0.0001,
    "optimizer

Running TensorBoard daemon on port 10000
Running ngrok daemon forwarding from port 10000
2020-05-27 09:49:20,994 sagemaker-containers ERROR    ExecuteUserScriptError:
Command "/opt/conda/bin/python -m train_sagemaker --aug_input_crop_size 512 --batch_size 16 --loss_weight_depth 0.4 --loss_weight_semseg 0.6 --model_name distillation --ngrok_auth_token 1afOlP4NIiVdFYkxUIrEctjBL0O_4NpsyRhTMoQt1zdTXugyN --ngrok_daemon_start True --optimizer adam --optimizer_lr 0.0001 --tensorboard_daemon_start True"
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, i

UnexpectedStatusException: Error for Training job Problem4-cropsize512-2020-05-27-09-41-34: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/opt/conda/bin/python -m train_sagemaker --aug_input_crop_size 512 --batch_size 16 --loss_weight_depth 0.4 --loss_weight_semseg 0.6 --model_name distillation --ngrok_auth_token 1afOlP4NIiVdFYkxUIrEctjBL0O_4NpsyRhTMoQt1zdTXugyN --ngrok_daemon_start True --optimizer adam --optimizer_lr 0.0001 --tensorboard_daemon_start True"
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understoo